# Getting Started with Granite Code

## Introduction


## Using a local model

## Using a remotely-hosted model

The Granite Code models are available on [Replicate](https://replicate.com/).

At the moment, they are only available to members of the Granite Code team.
Request an invite to get access.

This guide will demonstrate a basic inference call using the `replicate` package as well
as via LangChain.
In both cases, you will provide a [Replicate API Token](https://replicate.com/account/api-tokens).


In [1]:
import getpass, os

replicate_api_token = getpass.getpass()

os.environ["REPLICATE_API_TOKEN"] = replicate_api_token


We will keep the model constant through the guide as well.
Replicate distinguishes between a "deployment" of a model from a "model".
In this case, we want to specify the Granite Code development deployment.

In [2]:
deployment_id = "ibm/granite-dev"

model_id = "ibm/granite-8b-code-instruct:50da94a0b1b5d28e3161d1312077d856eb673b87e633438362e4820fed563444"

Let's also keep the prompt constant for now so we can focus on the inference calls:

In [3]:
prompt = "def f(x):"

### Replicate package

In [4]:
!pip install replicate


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import replicate

In [6]:
deployment = replicate.deployments.get(deployment_id)

prediction = deployment.predictions.create(
  input={"prompt": prompt}
)

prediction.wait()

print(prediction.output)


['System:\nYou are a helpful assistant\n\nQuestion:\ndef f(x):\n\nAnswer:\ndef f(x):\n    return x*x<|endoftext|>']


### Using Granite Code hosted on Replicate from LangChain

See https://python.langchain.com/v0.2/docs/integrations/providers/replicate/

See https://python.langchain.com/v0.2/docs/integrations/llms/replicate/

In [7]:
!pip install langchain_community


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
from langchain_community.llms import Replicate

model = Replicate(model=model_id,
                  model_kwargs={"max_length": 100},)

In [9]:
model(prompt)

/Users/anupamchakraborty/Code/granite-code-cookbook/.codecb_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'System:\nYou are a helpful assistant\n\nQuestion:\ndef f(x):\n\nAnswer:\ndef f(x):\n    return x*x<|endoftext|>'